<a href="https://www.kaggle.com/code/arsalannezhadfard/an-defects-datasets?scriptVersionId=146181526" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s3e23/sample_submission.csv
/kaggle/input/playground-series-s3e23/train.csv
/kaggle/input/playground-series-s3e23/test.csv


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder,LabelEncoder
from sklearn.metrics import classification_report,confusion_matrix,ConfusionMatrixDisplay
from scipy import stats
from sklearn.base import BaseEstimator, TransformerMixin
import scipy 

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
tr = pd.read_csv('/kaggle/input/playground-series-s3e23/train.csv')
ts = pd.read_csv('/kaggle/input/playground-series-s3e23/test.csv')

In [4]:
tr.head()

,id,loc,v(g),ev(g),iv(g),n,v,l,d,i,...,lOCode,lOComment,lOBlank,locCodeAndComment,uniq_Op,uniq_Opnd,total_Op,total_Opnd,branchCount,defects
0,0,22.0,3.0,1.0,2.0,60.0,278.63,0.06,19.56,14.25,...,17,1,1,0,16.0,9.0,38.0,22.0,5.0,False
1,1,14.0,2.0,1.0,2.0,32.0,151.27,0.14,7.00,21.11,...,11,0,1,0,11.0,11.0,18.0,14.0,3.0,False
2,2,11.0,2.0,1.0,2.0,45.0,197.65,0.11,8.05,22.76,...,8,0,1,0,12.0,11.0,28.0,17.0,3.0,False
3,3,8.0,1.0,1.0,1.0,23.0,94.01,0.19,5.25,17.86,...,4,0,2,0,8.0,6.0,16.0,7.0,1.0,True
4,4,11.0,2.0,1.0,2.0,17.0,60.94,0.18,5.63,12.44,...,7,0,2,0,7.0,6.0,10.0,10.0,3.0,False


In [5]:
tr.isna().sum()/tr.shape[0]

id                   0.0
loc                  0.0
v(g)                 0.0
ev(g)                0.0
iv(g)                0.0
n                    0.0
v                    0.0
l                    0.0
d                    0.0
i                    0.0
e                    0.0
b                    0.0
t                    0.0
lOCode               0.0
lOComment            0.0
lOBlank              0.0
locCodeAndComment    0.0
uniq_Op              0.0
uniq_Opnd            0.0
total_Op             0.0
total_Opnd           0.0
branchCount          0.0
defects              0.0
dtype: float64

In [6]:
tr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101763 entries, 0 to 101762
Data columns (total 23 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   id                 101763 non-null  int64  
 1   loc                101763 non-null  float64
 2   v(g)               101763 non-null  float64
 3   ev(g)              101763 non-null  float64
 4   iv(g)              101763 non-null  float64
 5   n                  101763 non-null  float64
 6   v                  101763 non-null  float64
 7   l                  101763 non-null  float64
 8   d                  101763 non-null  float64
 9   i                  101763 non-null  float64
 10  e                  101763 non-null  float64
 11  b                  101763 non-null  float64
 12  t                  101763 non-null  float64
 13  lOCode             101763 non-null  int64  
 14  lOComment          101763 non-null  int64  
 15  lOBlank            101763 non-null  int64  
 16  lo

In [7]:
tr=tr.drop(["id"],axis=1)

In [8]:
enc=LabelEncoder()
tr["defects_map"]=enc.fit_transform(tr["defects"])

In [9]:
ts

,id,loc,v(g),ev(g),iv(g),n,v,l,d,i,...,t,lOCode,lOComment,lOBlank,locCodeAndComment,uniq_Op,uniq_Opnd,total_Op,total_Opnd,branchCount
0,101763,33.0,5.0,1.0,4.0,144.0,824.82,0.04,26.96,30.05,...,1257.60,30,0,3,0,21.0,23.0,87.0,57.0,9.0
1,101764,27.0,8.0,8.0,2.0,125.0,646.24,0.04,22.82,27.22,...,804.58,23,0,2,0,18.0,19.0,70.0,49.0,15.0
2,101765,130.0,11.0,7.0,10.0,545.0,3831.40,0.02,48.15,66.17,...,6453.34,99,9,17,1,26.0,53.0,333.0,244.0,21.0
3,101766,65.0,7.0,1.0,7.0,156.0,855.71,0.06,17.23,49.89,...,896.42,45,8,10,0,15.0,26.0,88.0,60.0,13.0
4,101767,22.0,3.0,1.0,3.0,52.0,238.42,0.10,9.60,26.70,...,145.80,16,0,4,0,12.0,15.0,30.0,24.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67837,169600,41.0,1.0,1.0,1.0,125.0,656.55,0.07,14.00,47.61,...,479.35,31,0,7,0,14.0,27.0,71.0,54.0,1.0
67838,169601,20.0,3.0,1.0,3.0,38.0,161.42,0.15,6.75,23.28,...,58.94,15,0,3,0,9.0,10.0,23.0,15.0,5.0
67839,169602,24.0,2.0,1.0,2.0,52.0,240.00,0.11,9.00,26.67,...,120.00,18,0,4,0,16.0,16.0,36.0,18.0,3.0
67840,169603,18.0,2.0,1.0,1.0,49.0,216.64,0.11,9.33,24.02,...,117.68,12,0,2,0,8.0,12.0,29.0,22.0,3.0


In [10]:
x=tr.drop(["defects","defects_map"],axis=1)
y=tr["defects_map"]

In [11]:
x_test=ts.drop(["id"],axis=1)

## NAIVEBAYES

In [12]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score, GridSearchCV
nb_model = GaussianNB()
nb_scores = cross_val_score(nb_model, x, y, cv=5)
print(nb_scores)

nb_model.fit(x, y)
test_score_nb = nb_model.score(x, y)

print(f"Test score (accuracy): {test_score_nb}")

[0.79860463 0.79791677 0.80238785 0.79805425 0.798693  ]
Test score (accuracy): 0.7990330473747825


# Scaling

In [13]:
scaler=StandardScaler()
x_train_sc=scaler.fit_transform(x)
x_test_sc=scaler.transform(x_test)

## Logistic Regression

In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, GridSearchCV

# Logistic Regression with default parameters
#lr_model = LogisticRegression()
#lr_scores = cross_val_score(lr_model, x_train_sc, y, cv=5, scoring='accuracy')
#print(lr_scores)

"""
# Parameter grid for GridSearchCV
param_grid = {
    'penalty': ['l1', 'l2', 'elasticnet', 'none'],
    'C': [0.1, 1, 10],
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
    'max_iter': [100, 500, 1000]
}
lr_model=LogisticRegression()
grid_search_lr = GridSearchCV(lr_model, param_grid, scoring='accuracy', cv=5, n_jobs=-1, verbose=1)
grid_search_lr.fit(x_train_sc, y)
test_score_lr = grid_search_lr.best_estimator_.score(x_train_sc, y)

print(f"Best penalty value: {grid_search_lr.best_params_['penalty']}")
print(f"Best C value: {grid_search_lr.best_params_['C']}")
print(f"Best solver value: {grid_search_lr.best_params_['solver']}")
print(f"Best max_iter value: {grid_search_lr.best_params_['max_iter']}")
print(f"Best accuracy: {grid_search_lr.best_score_}")
"""

'\n# Parameter grid for GridSearchCV\nparam_grid = {\n    \'penalty\': [\'l1\', \'l2\', \'elasticnet\', \'none\'],\n    \'C\': [0.1, 1, 10],\n    \'solver\': [\'newton-cg\', \'lbfgs\', \'liblinear\', \'sag\', \'saga\'],\n    \'max_iter\': [100, 500, 1000]\n}\nlr_model=LogisticRegression()\ngrid_search_lr = GridSearchCV(lr_model, param_grid, scoring=\'accuracy\', cv=5, n_jobs=-1, verbose=1)\ngrid_search_lr.fit(x_train_sc, y)\ntest_score_lr = grid_search_lr.best_estimator_.score(x_train_sc, y)\n\nprint(f"Best penalty value: {grid_search_lr.best_params_[\'penalty\']}")\nprint(f"Best C value: {grid_search_lr.best_params_[\'C\']}")\nprint(f"Best solver value: {grid_search_lr.best_params_[\'solver\']}")\nprint(f"Best max_iter value: {grid_search_lr.best_params_[\'max_iter\']}")\nprint(f"Best accuracy: {grid_search_lr.best_score_}")\n'

In [15]:
# Logistic Regression with default parameters
lr_model = LogisticRegression(penalty="l1",C=1,solver="saga",max_iter=100)
lr_model.fit(x_train_sc,y)
lr_scores=cross_val_score(lr_model, x_train_sc, y, cv=5, scoring='accuracy')
print(lr_scores)

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[0.80931558 0.80906992 0.80951211 0.80704599 0.8081761 ]


/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


## Decision Tree

In [16]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV

# Decision Tree with default parameters
dt_model = DecisionTreeClassifier()
dt_model.fit(x,y)
dt_scores = cross_val_score(dt_model, x, y, cv=5, scoring='accuracy')
print(dt_scores)


[0.7239719  0.72166265 0.72185919 0.71717767 0.72356525]


## Random Forest

In [17]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV

# Random Forest with default parameters
rf_model = RandomForestClassifier()
rf_model.fit(x,y)
rf_scores = cross_val_score(rf_model, x, y, cv=5, scoring='accuracy')
print(rf_scores)

[0.80764506 0.80735027 0.80759593 0.80537539 0.80694772]


## Gradient Boosting

In [18]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV

# Gradient Boosting with default parameters
gb_model = GradientBoostingClassifier()
gb_model.fit(x,y)
gb_scores = cross_val_score(gb_model, x, y, cv=5, scoring='accuracy')
print(gb_scores)

[0.81643984 0.81545718 0.81481845 0.81254914 0.81554638]


## KNN 

In [19]:
from sklearn.neighbors import KNeighborsClassifier
knn_model = KNeighborsClassifier()
knn_model.fit(x_train_sc,y)
knn_scores = cross_val_score(knn_model, x_train_sc, y, cv=5, scoring='accuracy')
print(knn_scores)

[0.79138211 0.78863067 0.7918243  0.7900452  0.79250197]


## ANN-MLP

In [20]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV

# MLP with default parameters
mlp_model = MLPClassifier(random_state=42)
mlp_model.fit(x_train_sc,y)
mlp_scores = cross_val_score(mlp_model, x_train_sc, y, cv=5, scoring='accuracy')
print(mlp_scores)

/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


[0.81088783 0.81147742 0.81216528 0.81161557 0.81446541]


## DEEP Learning

In [21]:
from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping


val_percentage = 0.2

# Calculate the number of samples to use for validation
val_samples = int(val_percentage * len(x_train_sc))

# Split the data
X_val = x_train_sc[-val_samples:]
y_val = y[-val_samples:]





model = Sequential()

model.add(Dense(32, input_shape=(x_train_sc.shape[1],), activation='relu'))

model.add(Dense(16, activation='relu'))

model.add(Dense(8, activation='relu'))

model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

model.fit(x_train_sc, y, epochs=10, batch_size=32,validation_data=(X_val, y_val),callbacks=[early_stopping] )

Epoch 1/10
3181/3181 [==============================] - 20s 4ms/step - loss: 0.4408 - binary_accuracy: 0.8092 - val_loss: 0.4360 - val_binary_accuracy: 0.8124
Epoch 2/10
3181/3181 [==============================] - 12s 4ms/step - loss: 0.4323 - binary_accuracy: 0.8139 - val_loss: 0.4347 - val_binary_accuracy: 0.8137
Epoch 3/10
3181/3181 [==============================] - 13s 4ms/step - loss: 0.4315 - binary_accuracy: 0.8141 - val_loss: 0.4359 - val_binary_accuracy: 0.8129
Epoch 4/10
3181/3181 [==============================] - 12s 4ms/step - loss: 0.4308 - binary_accuracy: 0.8147 - val_loss: 0.4343 - val_binary_accuracy: 0.8132
Epoch 5/10
3181/3181 [==============================] - 12s 4ms/step - loss: 0.4306 - binary_accuracy: 0.8146 - val_loss: 0.4330 - val_binary_accuracy: 0.8145
Epoch 6/10
3181/3181 [==============================] - 13s 4ms/step - loss: 0.4302 - binary_accuracy: 0.8154 - val_loss: 0.4333 - val_binary_accuracy: 0.8154
Epoch 7/10
3181/3181 [========================

In [22]:

val_loss, val_accuracy = model.evaluate(X_val, y_val)

print(f'Validation Accuracy: {val_accuracy}')

636/636 [==============================] - 1s 2ms/step - loss: 0.4332 - binary_accuracy: 0.8153
Validation Accuracy: 0.8152515888214111


In [23]:
predicted = model.predict(x_test_sc)

submission_2=pd.DataFrame(predicted,index=ts["id"])


2121/2121 [==============================] - 3s 1ms/step


In [24]:
submission_2["defects"]=submission_2[0]

In [25]:
submission_2.drop([0],axis=1)

,defects
id,
101763,0.256603
101764,0.159876
101765,0.679282
101766,0.496401
101767,0.140080
...,...
169600,0.302556
169601,0.119022
169602,0.168763


In [26]:
submission=submission_2.to_csv('submission.csv',header=True, index=False)